## Оптимизация Pandas

**Прочитайте файл fines.csv, который вы сохранили в предыдущем упражнении**

In [1]:
import pandas as pd

fines = pd.read_csv('../data/fines.csv')
print(fines)

         CarNumber  Refund    Fines        Make    Model    Year
0     Y163O8161RUS       2   3200.0        Ford    Focus  1989.0
1      E432XX77RUS       1   6500.0      Toyota    Camry  1995.0
2      7184TT36RUS       1   2100.0        Ford    Focus  1984.0
3     X582HE161RUS       2   2000.0        Ford    Focus  2015.0
4     E34877152RUS       2   6100.0        Ford    Focus  2014.0
...            ...     ...      ...         ...      ...     ...
1131  1111C8197RUS       2    500.0      Toyota  Corolla  2007.0
1132  2222C8197RUS       1    100.0        Ford    Focus  1997.0
1133   33339Y50RUS       2  10100.0  Volkswagen     Golf  1984.0
1134  4444MM116RUS       1   2400.0       Skoda  Octavia  2017.0
1135  5555M7161RUS       2   7800.0      Porshe      911  2077.0

[1136 rows x 6 columns]


**Итерации: во всех следующих подзадачах необходимо рассчитать штрафы/возврат*год для каждой строки и создать новый столбец с рассчитанными данными и измерить время с помощью магической команды %%timeit в ячейке**
- цикл: написать функцию, которая проходит по фрейму данных, используя for i in range(0, len(df)), iloc и append() к списку, присвоить результат функции новому столбцу в фрейме данных

In [2]:
%%timeit
def loopfunc(df):
    calc = []
    for i in range(0, len(df)):
        calc.append(df['Fines'].iloc[i]/df['Refund'].iloc[i]*df['Year'].iloc[i])
    df['calc'] = calc
    
loopfunc(fines)

30.5 ms ± 1.74 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


- сделайте это с помощью iterrows()

In [3]:
%%timeit
def iterrows_func(df):
    calc = []
    for index, row in df.iterrows():
        calc.append(row['Fines'] / row['Refund'] * row['Year'])
    df['calc'] = calc
    
iterrows_func(fines)

71.9 ms ± 15.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


- сделайте это с помощью apply() и лямбда-функции

In [4]:
%%timeit
def apply_func(df):
    return df.apply(lambda row: row['Fines'] / row['Refund'] * row['Year'], axis=1)
    
fines['calc'] = apply_func(fines)

8.79 ms ± 511 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


- сделать это с помощью объектов Series из фрейма данных

In [5]:
%%timeit
def series_func(df):
    return df['Fines'] / df['Refund'] * df['Year']
    
fines['calc'] = series_func(fines)

263 µs ± 12.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


- сделать как в предыдущей подзадаче но с методом .values

In [6]:
%%timeit
def series_func(df):
    return df['Fines'].values / df['Refund'].values * df['Year'].values
    
fines['calc'] = series_func(fines)

97.6 µs ± 4.65 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


**Индексация: измерьте время с помощью магической команды %%timeit в ячейке**
- получить строку для определенного номера автомобиля, например, «O136HO197RUS»

In [7]:
%%timeit
car_row = fines[fines['CarNumber'] == 'O136HO197RUS']

296 µs ± 16.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


- установите индекс в вашем фрейме данных с помощью CarNumber

In [8]:
fines.set_index('CarNumber', inplace=True)

- снова, получив индекс для того же CarNumber

In [9]:
%%timeit
index_car_row = fines.loc['O136HO197RUS']

74.4 µs ± 4.43 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


**Нисходящее:**
- запустите df.info(memory_usage='deep'), обратите внимание на Dtype и использование памяти
- сделайте копию() вашего исходного фрейма данных в другой оптимизированный фрейм данных


  **Категории:**
 - измените столбцы типа объекта на категорию типа.
 - На этот раз проверьте использование памяти, возможно оно уменьшилось в 2-3 раза
 по сравнению с исходным кадром данных

In [10]:
fines.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 1136 entries, Y163O8161RUS to 5555M7161RUS
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Refund  1136 non-null   int64  
 1   Fines   1136 non-null   float64
 2   Make    1136 non-null   object 
 3   Model   1122 non-null   object 
 4   Year    930 non-null    float64
 5   calc    930 non-null    float64
dtypes: float64(3), int64(1), object(2)
memory usage: 281.0 KB


In [11]:
optimize_fines = fines.copy()
optimize_fines['Refund'] = pd.to_numeric(optimize_fines['Refund'], downcast='integer')
optimize_fines['Make'] = optimize_fines['Make'].astype('category')
optimize_fines['Model'] = optimize_fines['Model'].astype('category')
optimize_fines['Fines'] = optimize_fines['Fines'].astype('float32')
optimize_fines['Year'] = optimize_fines['Year'].astype('float32')
optimize_fines['calc'] = optimize_fines['calc'].astype('float32')

# сделали меньше, поменяв типы

print(optimize_fines.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
Index: 1136 entries, Y163O8161RUS to 5555M7161RUS
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   Refund  1136 non-null   int8    
 1   Fines   1136 non-null   float32 
 2   Make    1136 non-null   category
 3   Model   1122 non-null   category
 4   Year    930 non-null    float32 
 5   calc    930 non-null    float32 
dtypes: category(2), float32(3), int8(1)
memory usage: 127.0 KB
None


 **Очистка памяти**
 - С помощью %reset_selective и библиотеки gc очистите память вашего первоначального
 - Только фрейм данных

In [12]:
import gc
del_fines = fines.copy()
del del_fines
gc.collect()
%reset_selective -f del_fines
try:
    print("\nИнформация о памяти после очистки:")
    print(del_fines.info(memory_usage='deep'))
except NameError:
    print("Переменная del_fines была успешно удалена.")


Информация о памяти после очистки:
Переменная del_fines была успешно удалена.


In [14]:
optimized_df.info(memory_usage='deep')

NameError: name 'optimized_df' is not defined